# Notebook E-tivity 3 CE4021 Task 2

Student name: Ranganathan Deventhiran

Student ID: 23315695

<hr style=\"border:2px solid gray\"> </hr>

## Imports

If you believe required imports are missing, please contact your moderator.

<hr style=\"border:2px solid gray\"> </hr>

## Task 2

Use the below information to create a Naive Bayes SPAM filter. Test your filter using the messages in new_emails. You may add as many cells as you require to complete the task.

In [90]:
previous_spam = ['send us your password', 'review our website', 'send your password', 'send us your account']
previous_ham = ['Your activity report','benefits physical activity', 'the importance vows']
new_emails = {'spam':['renew your password', 'renew your vows'], 'ham':['benefits of our account', 'the importance of physical activity']}

##  Prior Probabilities:

The prior probabilities represent the probability of each class (spam or ham) occurring in the training dataset. 
To calculate the prior probabilities:
- Count the number of emails in the training set that belong to each class.
- Divide the count of emails in each class by the total number of emails in the training set

piror_spam= count(previous_spam) / (count(previous_spam) + count(previous_ham)) = 4/7 = 0.5714285714285714 <br>
prior_ham = count(previous_ham) / (count(previous_spam) + count(previous_ham)) = 3/7 = 0.42857142857142855

##  Conditional Probabilities:
The conditional probabilities represent the likelihood of each feature given a specific class (spam or ham). <br>
To calculate the conditional probabilities:<br>
- For each feature, count the occurrences of the each word  in spam emails.
- divide it by the total count of all words in spam emails
- Laplace Smoothing :
<t><t><p>In situations where a feature does not appear in the training set for a particular class, the conditional probability becomes zero, which can lead to issues when calculating posterior probabilities. To mitigate this problem, we add 1 to the counts of all features in both classes and increase the total count of words in the vocabulary by the number of unique words</p>

|   word                 |                  probability     |
|------------------------|--------------------------------- |              
|p(send/SPAM)            |              0.18181818181818182 |           
|p(us/SPAM)              |              0.13636363636363635 |          
|p(your/SPAM)            |              0.18181818181818182 |          
|p(password/SPAM)        |              0.13636363636363635 |          
|p(review/SPAM)          |              0.09090909090909091 |          
|p(our/SPAM)             |              0.09090909090909091 |          
|p(website/SPAM)         |              0.09090909090909091 |          
|p(account/SPAM)         |              0.09090909090909091 |
|p(your/HAM)             |              0.11764705882352941 |
|p(actity/HAM)           |              0.11764705882352941 |
|p(report/HAM)           |              0.11764705882352941 |
|p(benefits/HAM)         |              0.11764705882352941 |
|p(physical/HAM)         |              0.11764705882352941 |
|p(the/HAM)              |              0.11764705882352941 |
|p(importance/HAM)       |              0.11764705882352941 |
|p(vows/HAM)             |              0.11764705882352941 |

## Naive Bayes Classifier


In [91]:
class SpamOrHamFilter:
    """ 
       SpamOrHamFilter class is used to filter incoming mail for SPAM or ham using
       Naive Bayes Classifier 
    """
    def __init__(self):
        self.prior_spam = 0.0
        self.prior_ham = 0.0
        self.spam_word_probs = {}
        self.ham_word_probs = {}

    def prep_word_count_dict(self,list_of_email):
        """ 
        this method takes the list of emails and create the work dictionary with each unique word as key and 
        the occurences of this word in all the email as value
        """
        word_dict={}
        total_words=0
        for email in list_of_email:
            words = email.lower().split() 
            for word in words:               
                word_dict[word] = word_dict.get(word, 0) + 1  
                total_words += 1
        return total_words,word_dict

    def calculate_probabiltiy(self, previous_spam,previous_ham):
        """ 
        this method takes word dictionaries created by prep_word_count_dict method for both spam and ham email lists
        calcualte the prior probabilities and word probabilities
        count the occurrences of the each word  in spam emails and divide it by the total count of all words in spam emails
        """
        spam_emails = len(previous_spam)
        ham_emails = len(previous_ham)
        total_emails = spam_emails + ham_emails

        self.prior_spam = spam_emails / total_emails #p(spam)
        self.prior_ham = ham_emails / total_emails  #p(ham)

      

        self.spam_total_words,spam_word_counts_dict = self.prep_word_count_dict(previous_spam)

        self.ham_total_words,ham_word_counts_dict = self.prep_word_count_dict(previous_ham)
        for word, count in spam_word_counts_dict.items():
            #Using Laplace smoothing, we add 1 to the counts of all features in both classes and 
            #increase the total count of words in the vocabulary by the number of unique words (len(spam_word_counts_dict)).
            self.spam_word_probs[word] = (count + 1) / (self.spam_total_words + len(spam_word_counts_dict)) # Laplace smoothing 
            #self.spam_word_probs[word] = (count + 1) / (spam_total_words + 2) #other smooting 
            #self.spam_word_probs[word] = (count ) / (self.spam_total_words) # without smooting  
      
                   

        for word, count in ham_word_counts_dict.items():
            self.ham_word_probs[word] = (count + 1) / (self.ham_total_words + len(ham_word_counts_dict)) # Laplace smoothing 

    def is_ham_or_spam(self, email):  
        """ 
        for the given email, this method calculates the posterior probabilities for both spam and ham
        and decides whether this email is spam or ham
        """      
        words = email.lower().split()
        spam_score = self.prior_spam
        ham_score = self.prior_ham

        for word in words:           
            spam_prob = self.spam_word_probs.get(word, 1 / (sum(self.spam_word_probs.values()) + len(self.spam_word_probs)))
            ham_prob = self.ham_word_probs.get(word, 1 / (sum(self.ham_word_probs.values()) + len(self.ham_word_probs)))
            spam_score *= spam_prob
            ham_score *= ham_prob   

        if spam_score > ham_score:
            print("{:<38} {:<30} {:<30} {:<15}".format(email, spam_score, ham_score,"spam" ))
            return "spam"
        else:
            print("{:<38} {:<30} {:<30} {:<15}".format(email, spam_score, ham_score,"ham" ))

            return "ham"
    
    def predict(self, new_emails : dict):
        """ 
        Predict method iterate the new_emails
        and checks wether the email is spam or ham and calculate the accurecy of predictions.
        """   
        success = 0
        total_emails =0
        print("{:<38} {:<30} {:<30} {:<15}".format("Email", "SpamScore", "HamScore" , "Classification"))

        keys= list(new_emails.keys())
        for key in keys:
            list_of_emails=new_emails[key]
            for email in list_of_emails:
                total_emails += 1
                if self.is_ham_or_spam(email) == key:
                    success += 1
        
        print("Accurecy of prediction(%) is :",(success/total_emails) *100)

In [92]:
previous_spam = ['send us your password', 'review our website', 'send your password', 'send us your account']
previous_ham = ['Your activity report','benefits physical activity', 'the importance vows']
new_emails = {'spam':['renew your password', 'renew your vows'], 'ham':['benefits of our account', 'the importance of physical activity']}

spamOrHamFilter=SpamOrHamFilter()
spamOrHamFilter.calculate_probabiltiy(previous_spam,previous_ham)
print("Ham Or Spam Predictions:")
spamOrHamFilter.predict(new_emails)

Ham Or Spam Predictions:
Email                                  SpamScore                      HamScore                       Classification 
renew your password                    0.0015741833923652105          0.0006224712107065047          spam           
renew your vows                        0.0012826679493346158          0.0006590871642774756          spam           
benefits of our account                5.8303088606118916e-05         6.91634678562783e-05           ham            
the importance of physical activity    9.677193033388733e-06          1.3683470538632712e-05         ham            
Accurecy of prediction(%) is : 100.0


<hr style=\"border:2px solid gray\"> </hr>

## Reflection

Write you reflection in below cell.